In [1]:
# Dependencies

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
import numpy as np
import tensorflow as tf
import nltk
from collections import Counter
import string
import itertools
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import tensorflow_text as text
import tensorflow_hub as hub
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import GlobalMaxPool1D, BatchNormalization, Dense, RNN, GRU, LSTM, TimeDistributed, Bidirectional, Activation, Embedding, Input, Conv1D, Dropout
import tensorflow as tf
import keras.backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from scipy.special import softmax
from transformers import pipeline
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments
from tensorflow.keras import mixed_precision
from tensorflow.keras.utils import to_categorical
from datasets import Dataset

# tf.config.set_visible_devices([], 'GPU')
plt.style.use('ggplot')

# Define mixed precision policy
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

from datasets import DatasetDict

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3070, compute capability 8.6


In [2]:
# Data imports
with open('reviews.txt', 'r') as f:
  reviews = f.read()
with open('labels.txt', 'r') as f:
  labels = f.read()
  
  
reviews = reviews.split('\n')
labels = labels.split('\n')
labels = [1 if label == "positive" else 0 for label in labels]

reviews = reviews[:10000]
labels = labels[:10000]

In [3]:
df = pd.DataFrame({'text': reviews, 'label': labels})
df.head(3)

,text,label
0,bromwell high is a cartoon comedy . it ran at ...,1
1,story of a man who has unnatural feelings for ...,0
2,homelessness or houselessness as george carli...,1


In [4]:
from datasets import DatasetDict

ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['text', 'label'],
    num_rows: 10000
})

In [5]:
train_test_valid = ds.train_test_split()
test_valid = train_test_valid['test'].train_test_split()

train_test_valid_dataset = DatasetDict({
  'train':train_test_valid['train'],
  'test':test_valid['test'],
  'valid':test_valid['train']
})

dataset = train_test_valid_dataset

In [6]:
checkpoint = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [7]:
def tokenize_function(train_dataset):
    return tokenizer(train_dataset['text'], padding='max_length', truncation=True, max_length = 200) 
  

tokenized_dataset = dataset.map(tokenize_function, batched=True)

tokenized_dataset

train_dataset = tokenized_dataset['train']
eval_dataset = tokenized_dataset['valid']
test_dataset = tokenized_dataset['test']

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

Map:   0%|          | 0/625 [00:00<?, ? examples/s]

Map:   0%|          | 0/1875 [00:00<?, ? examples/s]

In [8]:
train_set = train_dataset.remove_columns(['text']).with_format('tensorflow')

tf_eval_dataset = eval_dataset.remove_columns(['text']).with_format('tensorflow')

tf_test_dataset = test_dataset.remove_columns(['text']).with_format('tensorflow')

In [9]:
train_features = { x: train_set[x] for x in tokenizer.model_input_names  }

train_set_for_final_model = tf.data.Dataset.from_tensor_slices((train_features, train_set['label'] ))

train_set_for_final_model = train_set_for_final_model.shuffle(len(train_set)).batch(2)


eval_features = {x: tf_eval_dataset[x] for x in tokenizer.model_input_names}
val_set_for_final_model = tf.data.Dataset.from_tensor_slices((eval_features, tf_eval_dataset["label"]))
val_set_for_final_model = val_set_for_final_model.batch(2)

test_features = {x: tf_test_dataset[x] for x in tokenizer.model_input_names}
test_set_for_final_model = tf.data.Dataset.from_tensor_slices((test_features, tf_test_dataset["label"]))
test_set_for_final_model = test_set_for_final_model.batch(2)

In [10]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
model.layers[0].trainable = False


model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
history = model.fit(train_set_for_final_model, validation_data=val_set_for_final_model, epochs=3, batch_size = 2)

Epoch 1/3
3750/3750 [==============================] - 263s 67ms/step - loss: 0.6814 - accuracy: 0.5603 - val_loss: 0.6634 - val_accuracy: 0.6149
Epoch 2/3
3750/3750 [==============================] - 254s 68ms/step - loss: 0.6610 - accuracy: 0.6071 - val_loss: 0.6492 - val_accuracy: 0.6336
Epoch 3/3
3750/3750 [==============================] - 258s 69ms/step - loss: 0.6480 - accuracy: 0.6300 - val_loss: 0.6350 - val_accuracy: 0.6533


In [12]:
train_set['label']

<tf.Tensor: shape=(7500,), dtype=int64, numpy=array([1, 0, 1, ..., 0, 0, 0], dtype=int64)>